In [1]:
import pandas as pd
import numpy as np
import os
import json
import itertools as it
import time
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from datetime import date, datetime, timedelta
from tensorflow import keras
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings("ignore")

2024-01-24 08:19:08.112141: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-24 08:19:08.360462: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 08:19:09.183317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0
majestic

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,google.com,0,103,111,111,103,108,101,46,99,...,0,0,0,0,0,0,0,0,0,0
1,facebook.com,0,102,97,99,101,98,111,111,107,...,0,0,0,0,0,0,0,0,0,0
2,youtube.com,0,121,111,117,116,117,98,101,46,...,0,0,0,0,0,0,0,0,0,0
3,twitter.com,0,116,119,105,116,116,101,114,46,...,0,0,0,0,0,0,0,0,0,0
4,instagram.com,0,105,110,115,116,97,103,114,97,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,fidgit.com,0,102,105,100,103,105,116,46,99,...,0,0,0,0,0,0,0,0,0,0
999996,fixwindows10connections.com,0,102,105,120,119,105,110,100,111,...,0,0,0,0,0,0,0,0,0,0
999997,flyerguide.com,0,102,108,121,101,114,103,117,105,...,0,0,0,0,0,0,0,0,0,0
999998,forwardpathway.com,0,102,111,114,119,97,114,100,112,...,0,0,0,0,0,0,0,0,0,0


In [10]:
dga = pd.read_csv('./treino/df_train.csv')
dga = dga.drop('family', axis = 1)
dga

,domain,label,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
0,uzwdmrlhr.tk,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
1,uzwdmrlhr.com,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
2,uzwdmrlhr.pages.dev,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
3,rhrlwdumz.tk,1,114,104,114,108,119,100,117,109,...,0,0,0,0,0,0,0,0,0,0
4,rhrlwdumz.com,1,114,104,114,108,119,100,117,109,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352254,kwgjjpbcaaanbivklyxx.com,1,107,119,103,106,106,112,98,99,...,0,0,0,0,0,0,0,0,0,0
1352255,vxijfvgvihjnfvofxmyc.com,1,118,120,105,106,102,118,103,118,...,0,0,0,0,0,0,0,0,0,0
1352256,cepswmbgtknhtkhlsecg.com,1,99,101,112,115,119,109,98,103,...,0,0,0,0,0,0,0,0,0,0
1352257,tifjhjtouqmdlbkfnuow.com,1,116,105,102,106,104,106,116,111,...,0,0,0,0,0,0,0,0,0,0


In [11]:
dga.columns = majestic.columns

In [12]:
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

In [13]:
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state = 1)], axis = 0, ignore_index=True)
df_treino

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,wwoof.fr,0,119,119,111,111,102,46,102,114,...,0,0,0,0,0,0,0,0,0,0
1,digimanthan.com,0,100,105,103,105,109,97,110,116,...,0,0,0,0,0,0,0,0,0,0
2,injo.com,0,105,110,106,111,46,99,111,109,...,0,0,0,0,0,0,0,0,0,0
3,aclanthology.info,0,97,99,108,97,110,116,104,111,...,0,0,0,0,0,0,0,0,0,0
4,aseabreeze.co.uk,0,97,115,101,97,98,114,101,101,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,h12cz81gp9t9s222dpro2n8la.com,1,104,49,50,99,122,56,49,103,...,0,0,0,0,0,0,0,0,0,0
999996,ykokieyqwseggkeq.xyz,1,121,107,111,107,105,101,121,113,...,0,0,0,0,0,0,0,0,0,0
999997,aefhjmajhhjr.bazar,1,97,101,102,104,106,109,97,106,...,0,0,0,0,0,0,0,0,0,0
999998,pkrgsrfiskhg.com,1,112,107,114,103,115,114,102,105,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

In [15]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

In [16]:
X_train.shape

(1000000, 70)

In [17]:
#Carregando hyperparâmetros de regularizadores, otimizador e função de ativação
l1l2 = keras.regularizers.l1_l2(l1=0.00001, l2=0.0001)
opt = keras.optimizers.Adam(learning_rate=0.0001)
fx = "relu"

In [18]:
#Estrutura do modelo convolucional profundo
model_bin = tf.keras.Sequential([
      
        layers.Embedding(input_dim=256,output_dim=60,input_length=70),
    
        layers.Conv1D(500, 7, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
            
        layers.Conv1D(250, 6, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=6, strides=1, padding='same'),
    
        layers.Conv1D(120, 5, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
    
        layers.Conv1D(60, 5, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=4, strides=1, padding='same'),
    
        layers.Conv1D(30, 4, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
    
        layers.Conv1D(30, 3, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=3, strides=1, padding='same'),
    
        layers.Flatten(),
        layers.Dense(15, activation=fx, activity_regularizer=l1l2),
        layers.Dense(1, activation='sigmoid')])

model_bin.compile(optimizer = opt, loss = 'binary_crossentropy', metrics='accuracy')
model_bin.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 60)            15360     
                                                                 
 conv1d (Conv1D)             (None, 70, 500)           210500    
                                                                 
 re_lu (ReLU)                (None, 70, 500)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 70, 250)           750250    
                                                                 
 re_lu_1 (ReLU)              (None, 70, 250)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 70, 250)           0         
 D)                                                              
                                                        

In [19]:
best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

In [20]:
# Ajustando uso de memoria RAM e GPU
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

In [21]:
model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.1246 - accuracy: 0.9548INFO:tensorflow:Assets written to: best_model_acc_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.1246 - accuracy: 0.9548 - val_loss: 0.1063 - val_accuracy: 0.9615
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0697 - accuracy: 0.9765INFO:tensorflow:Assets written to: best_model_acc_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_balance/assets


4000/4000 [==============================] - 162s 41ms/step - loss: 0.0697 - accuracy: 0.9765 - val_loss: 0.0830 - val_accuracy: 0.9712
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0549 - accuracy: 0.9816INFO:tensorflow:Assets written to: best_model_acc_balance/assets


...

4000/4000 [==============================] - 162s 41ms/step - loss: 0.0207 - accuracy: 0.9938 - val_loss: 0.1170 - val_accuracy: 0.9671
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0177 - accuracy: 0.9949INFO:tensorflow:Assets written to: best_model_acc_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0177 - accuracy: 0.9949 - val_loss: 0.1373 - val_accuracy: 0.9630
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0154 - accuracy: 0.9955INFO:tensorflow:Assets written to: best_model_acc_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0154 - accuracy: 0.9955 - val_loss: 0.1158 - val_accuracy: 0.9688


In [16]:
model_bin.load_weights('best_model_acc_balance')

2024-01-22 16:52:40.854717: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open best_model_acc: FAILED_PRECONDITION: best_model_acc; É um diretório: perhaps your file is in a different file format and you need to use a different restore operator?


In [22]:
model_bin = tf.keras.models.load_model('best_model_acc_balance')

In [23]:
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

## Testes Treinamento Inicial

In [25]:
model_bin = tf.keras.models.load_model('best_model_acc_balance')

for i in range (0,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga.sample(500000, random_state = 1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 385s 12ms/step
Matriz de Confusão Teste:  0
[[490650   9350]
 [ 15423 484577]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9695    0.9813    0.9754    500000
           1     0.9811    0.9692    0.9751    500000

    accuracy                         0.9752   1000000
   macro avg     0.9753    0.9752    0.9752   1000000
weighted avg     0.9753    0.9752    0.9752   1000000

____________________
31250/31250 [==============================] - 385s 12ms/step
Matriz de Confusão Teste:  1
[[490650   9350]
 [ 17071 482929]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9664    0.9813    0.9738    500000
           1     0.9810    0.9659    0.9734    500000

    accuracy                         0.9736   1000000
   macro avg     0.9737    0.9736    0.9736   1000000
weighted avg     0.9737    0.9736    0.9736   1000000

___________________

## Teste Primeiro Treinamento Incremental

In [26]:
model_bin = tf.keras.models.load_model('best_model_acc_balance')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste0.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase0_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0396 - accuracy: 0.9875INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0396 - accuracy: 0.9875 - val_loss: 0.0786 - val_accuracy: 0.9720
Epoch 2/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0277 - accuracy: 0.9917INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


4000/4000 [==============================] - 162s 41ms/step - loss: 0.0277 - accuracy: 0.9917 - val_loss: 0.0708 - val_accuracy: 0.9757
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0213 - accuracy: 0.9938INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


...

4000/4000 [==============================] - 161s 40ms/step - loss: 0.0102 - accuracy: 0.9973 - val_loss: 0.0922 - val_accuracy: 0.9774
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0091 - accuracy: 0.9976INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0091 - accuracy: 0.9976 - val_loss: 0.1491 - val_accuracy: 0.9654
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0086 - accuracy: 0.9977INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0086 - accuracy: 0.9977 - val_loss: 0.1169 - val_accuracy: 0.9721


In [28]:
model_bin = tf.keras.models.load_model('best_model_acc_fase0_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (1,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 390s 12ms/step
Matriz de Confusão Teste:  1
[[490819   9181]
 [ 14206 485794]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9719    0.9816    0.9767    500000
           1     0.9815    0.9716    0.9765    500000

    accuracy                         0.9766   1000000
   macro avg     0.9767    0.9766    0.9766   1000000
weighted avg     0.9767    0.9766    0.9766   1000000

____________________
31250/31250 [==============================] - 396s 13ms/step
Matriz de Confusão Teste:  2
[[490819   9181]
 [ 14663 485337]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9710    0.9816    0.9763    500000
           1     0.9814    0.9707    0.9760    500000

    accuracy                         0.9762   1000000
   macro avg     0.9762    0.9762    0.9762   1000000
weighted avg     0.9762    0.9762    0.9762   1000000

___________________

## Teste Segundo Treinamento Incremental

In [29]:
model_bin = tf.keras.models.load_model('best_model_acc_fase0_balance')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste1.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase1_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0326 - accuracy: 0.9902INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


4000/4000 [==============================] - 165s 41ms/step - loss: 0.0326 - accuracy: 0.9902 - val_loss: 0.0552 - val_accuracy: 0.9810
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0215 - accuracy: 0.9940INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0215 - accuracy: 0.9940 - val_loss: 0.0480 - val_accuracy: 0.9860
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0161 - accuracy: 0.9957INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


...

4000/4000 [==============================] - 163s 41ms/step - loss: 0.0078 - accuracy: 0.9980 - val_loss: 0.1112 - val_accuracy: 0.9747
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0072 - accuracy: 0.9982INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


4000/4000 [==============================] - 163s 41ms/step - loss: 0.0072 - accuracy: 0.9982 - val_loss: 0.0995 - val_accuracy: 0.9767
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0069 - accuracy: 0.9983INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase1_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0069 - accuracy: 0.9983 - val_loss: 0.1237 - val_accuracy: 0.9731


In [30]:
model_bin = tf.keras.models.load_model('best_model_acc_fase1_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (2,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 397s 13ms/step
Matriz de Confusão Teste:  2
[[491372   8628]
 [ 13315 486685]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9736    0.9827    0.9782    500000
           1     0.9826    0.9734    0.9780    500000

    accuracy                         0.9781   1000000
   macro avg     0.9781    0.9781    0.9781   1000000
weighted avg     0.9781    0.9781    0.9781   1000000

____________________
31250/31250 [==============================] - 381s 12ms/step
Matriz de Confusão Teste:  3
[[491372   8628]
 [ 15811 484189]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9688    0.9827    0.9757    500000
           1     0.9825    0.9684    0.9754    500000

    accuracy                         0.9756   1000000
   macro avg     0.9757    0.9756    0.9756   1000000
weighted avg     0.9757    0.9756    0.9756   1000000

___________________

## Teste Terceiro Treino Incremental

In [31]:
model_bin = tf.keras.models.load_model('best_model_acc_fase1_balance')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste2.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase2_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0278 - accuracy: 0.9920INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


4000/4000 [==============================] - 165s 41ms/step - loss: 0.0278 - accuracy: 0.9920 - val_loss: 0.0485 - val_accuracy: 0.9840
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0172 - accuracy: 0.9955INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0172 - accuracy: 0.9955 - val_loss: 0.0582 - val_accuracy: 0.9834
Epoch 3/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0123 - accuracy: 0.9969INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


...

4000/4000 [==============================] - 163s 41ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 0.1014 - val_accuracy: 0.9789
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0059 - accuracy: 0.9986INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


4000/4000 [==============================] - 162s 41ms/step - loss: 0.0059 - accuracy: 0.9986 - val_loss: 0.0980 - val_accuracy: 0.9796
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0056 - accuracy: 0.9987INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2_balance/assets


4000/4000 [==============================] - 162s 41ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 0.1362 - val_accuracy: 0.9735


In [32]:
model_bin = tf.keras.models.load_model('best_model_acc_fase2_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (3,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 398s 13ms/step
Matriz de Confusão Teste:  3
[[491458   8542]
 [ 15065 484935]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9703    0.9829    0.9765    500000
           1     0.9827    0.9699    0.9762    500000

    accuracy                         0.9764   1000000
   macro avg     0.9765    0.9764    0.9764   1000000
weighted avg     0.9765    0.9764    0.9764   1000000

____________________
31250/31250 [==============================] - 382s 12ms/step
Matriz de Confusão Teste:  4
[[491458   8542]
 [ 15141 484859]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9701    0.9829    0.9765    500000
           1     0.9827    0.9697    0.9762    500000

    accuracy                         0.9763   1000000
   macro avg     0.9764    0.9763    0.9763   1000000
weighted avg     0.9764    0.9763    0.9763   1000000

___________________

## Teste Quarto Treino Incremental

In [34]:
model_bin = tf.keras.models.load_model('best_model_acc_fase2_balance')

del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0
del(dga)
dga = pd.read_csv('./teste/df_teste3.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase3_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0275 - accuracy: 0.9922INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0275 - accuracy: 0.9922 - val_loss: 0.0425 - val_accuracy: 0.9859
Epoch 2/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0162 - accuracy: 0.9959INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0162 - accuracy: 0.9959 - val_loss: 0.0457 - val_accuracy: 0.9863
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0117 - accuracy: 0.9971INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


...

4000/4000 [==============================] - 161s 40ms/step - loss: 0.0059 - accuracy: 0.9986 - val_loss: 0.0770 - val_accuracy: 0.9836
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0054 - accuracy: 0.9987INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0054 - accuracy: 0.9987 - val_loss: 0.0866 - val_accuracy: 0.9816
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0052 - accuracy: 0.9988INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0052 - accuracy: 0.9988 - val_loss: 0.0859 - val_accuracy: 0.9815


In [35]:
model_bin = tf.keras.models.load_model('best_model_acc_fase3_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (4,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    
    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 397s 13ms/step
Matriz de Confusão Teste:  4
[[487104  12896]
 [  9728 490272]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9804    0.9742    0.9773    500000
           1     0.9744    0.9805    0.9774    500000

    accuracy                         0.9774   1000000
   macro avg     0.9774    0.9774    0.9774   1000000
weighted avg     0.9774    0.9774    0.9774   1000000

____________________
31250/31250 [==============================] - 384s 12ms/step
Matriz de Confusão Teste:  5
[[487104  12896]
 [ 10574 489426]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9788    0.9742    0.9765    500000
           1     0.9743    0.9789    0.9766    500000

    accuracy                         0.9765   1000000
   macro avg     0.9765    0.9765    0.9765   1000000
weighted avg     0.9765    0.9765    0.9765   1000000

___________________

## Teste Quinto Treino Incremental

In [36]:
model_bin = tf.keras.models.load_model('best_model_acc_fase3_balance')
del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

del(dga)
dga = pd.read_csv('./teste/df_teste4.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase4_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0242 - accuracy: 0.9933INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0242 - accuracy: 0.9933 - val_loss: 0.0405 - val_accuracy: 0.9869
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0141 - accuracy: 0.9966INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0141 - accuracy: 0.9966 - val_loss: 0.0457 - val_accuracy: 0.9877
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0099 - accuracy: 0.9977INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


...

4000/4000 [==============================] - 161s 40ms/step - loss: 0.0052 - accuracy: 0.9988 - val_loss: 0.0852 - val_accuracy: 0.9827
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0047 - accuracy: 0.9990INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


4000/4000 [==============================] - 162s 41ms/step - loss: 0.0047 - accuracy: 0.9990 - val_loss: 0.0910 - val_accuracy: 0.9822
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0047 - accuracy: 0.9990INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0047 - accuracy: 0.9990 - val_loss: 0.0873 - val_accuracy: 0.9829


In [37]:
model_bin = tf.keras.models.load_model('best_model_acc_fase4_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (5,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns

    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 400s 13ms/step
Matriz de Confusão Teste:  5
[[487617  12383]
 [ 10104 489896]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9797    0.9752    0.9775    500000
           1     0.9753    0.9798    0.9776    500000

    accuracy                         0.9775   1000000
   macro avg     0.9775    0.9775    0.9775   1000000
weighted avg     0.9775    0.9775    0.9775   1000000

____________________
31250/31250 [==============================] - 380s 12ms/step
Matriz de Confusão Teste:  6
[[487617  12383]
 [  9861 490139]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9802    0.9752    0.9777    500000
           1     0.9754    0.9803    0.9778    500000

    accuracy                         0.9778   1000000
   macro avg     0.9778    0.9778    0.9778   1000000
weighted avg     0.9778    0.9778    0.9778   1000000

___________________

## Teste Sexto Treino Incremental

In [38]:
model_bin = tf.keras.models.load_model('best_model_acc_fase4_balance')
del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

del(dga)
dga = pd.read_csv('./teste/df_teste5.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase5_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0242 - accuracy: 0.9935INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0242 - accuracy: 0.9935 - val_loss: 0.0437 - val_accuracy: 0.9862
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0143 - accuracy: 0.9966INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0143 - accuracy: 0.9966 - val_loss: 0.0502 - val_accuracy: 0.9856
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0102 - accuracy: 0.9976INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


...

4000/4000 [==============================] - 161s 40ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.0849 - val_accuracy: 0.9823
Epoch 8/10
4000/4000 [==============================] - 160s 40ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.0682 - val_accuracy: 0.9852
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0047 - accuracy: 0.9989INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0047 - accuracy: 0.9989 - val_loss: 0.0857 - val_accuracy: 0.9834
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0045 - accuracy: 0.9990INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0045 - accuracy: 0.9990 - val_loss: 0.0965 - val_accuracy: 0.9820


In [39]:
model_bin = tf.keras.models.load_model('best_model_acc_fase5_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (6,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns

    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 396s 13ms/step
Matriz de Confusão Teste:  6
[[487722  12278]
 [  8793 491207]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9823    0.9754    0.9789    500000
           1     0.9756    0.9824    0.9790    500000

    accuracy                         0.9789   1000000
   macro avg     0.9790    0.9789    0.9789   1000000
weighted avg     0.9790    0.9789    0.9789   1000000

____________________
31250/31250 [==============================] - 378s 12ms/step
Matriz de Confusão Teste:  7
[[487722  12278]
 [  9986 490014]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9799    0.9754    0.9777    500000
           1     0.9756    0.9800    0.9778    500000

    accuracy                         0.9777   1000000
   macro avg     0.9777    0.9777    0.9777   1000000
weighted avg     0.9777    0.9777    0.9777   1000000

___________________

## Sétimo Treino Incremental

In [40]:
model_bin = tf.keras.models.load_model('best_model_acc_fase5_balance')
del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

del(dga)
dga = pd.read_csv('./teste/df_teste6.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase6_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0216 - accuracy: 0.9944INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


4000/4000 [==============================] - 164s 40ms/step - loss: 0.0216 - accuracy: 0.9944 - val_loss: 0.0414 - val_accuracy: 0.9882
Epoch 2/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0126 - accuracy: 0.9971INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0126 - accuracy: 0.9971 - val_loss: 0.0441 - val_accuracy: 0.9881
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0093 - accuracy: 0.9979INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


...

4000/4000 [==============================] - 161s 40ms/step - loss: 0.0045 - accuracy: 0.9990 - val_loss: 0.0856 - val_accuracy: 0.9844
Epoch 9/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0043 - accuracy: 0.9991INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 0.0906 - val_accuracy: 0.9833
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0041 - accuracy: 0.9991INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 0.0883 - val_accuracy: 0.9838


In [41]:
model_bin = tf.keras.models.load_model('best_model_acc_fase6_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (7,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns

    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 391s 13ms/step
Matriz de Confusão Teste:  7
[[487019  12981]
 [  8725 491275]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9824    0.9740    0.9782    500000
           1     0.9743    0.9826    0.9784    500000

    accuracy                         0.9783   1000000
   macro avg     0.9783    0.9783    0.9783   1000000
weighted avg     0.9783    0.9783    0.9783   1000000

____________________
31250/31250 [==============================] - 376s 12ms/step
Matriz de Confusão Teste:  8
[[487019  12981]
 [  8590 491410]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9827    0.9740    0.9783    500000
           1     0.9743    0.9828    0.9785    500000

    accuracy                         0.9784   1000000
   macro avg     0.9785    0.9784    0.9784   1000000
weighted avg     0.9785    0.9784    0.9784   1000000

___________________

## Oitavo Treino Incremental

In [42]:
model_bin = tf.keras.models.load_model('best_model_acc_fase6_balance')
del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

del(dga)
dga = pd.read_csv('./teste/df_teste7.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase7_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0220 - accuracy: 0.9942INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0220 - accuracy: 0.9942 - val_loss: 0.0344 - val_accuracy: 0.9897
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0130 - accuracy: 0.9969INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0130 - accuracy: 0.9969 - val_loss: 0.0346 - val_accuracy: 0.9900
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0090 - accuracy: 0.9980INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


...

4000/4000 [==============================] - 162s 41ms/step - loss: 0.0044 - accuracy: 0.9991 - val_loss: 0.0663 - val_accuracy: 0.9868
Epoch 9/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0042 - accuracy: 0.9991INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 0.0714 - val_accuracy: 0.9862
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0040 - accuracy: 0.9992INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase7_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0040 - accuracy: 0.9992 - val_loss: 0.0962 - val_accuracy: 0.9824


In [43]:
model_bin = tf.keras.models.load_model('best_model_acc_fase7_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (8,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns

    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 400s 13ms/step
Matriz de Confusão Teste:  8
[[488145  11855]
 [  8879 491121]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9821    0.9763    0.9792    500000
           1     0.9764    0.9822    0.9793    500000

    accuracy                         0.9793   1000000
   macro avg     0.9793    0.9793    0.9793   1000000
weighted avg     0.9793    0.9793    0.9793   1000000

____________________
31250/31250 [==============================] - 385s 12ms/step
Matriz de Confusão Teste:  9
[[488145  11855]
 [ 10357 489643]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9792    0.9763    0.9778    500000
           1     0.9764    0.9793    0.9778    500000

    accuracy                         0.9778   1000000
   macro avg     0.9778    0.9778    0.9778   1000000
weighted avg     0.9778    0.9778    0.9778   1000000

___________________

## Nono Treino Incremental

In [44]:
model_bin = tf.keras.models.load_model('best_model_acc_fase7_balance')
del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

del(dga)
dga = pd.read_csv('./teste/df_teste8.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase8_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0202 - accuracy: 0.9948INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0202 - accuracy: 0.9948 - val_loss: 0.0347 - val_accuracy: 0.9903
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0115 - accuracy: 0.9974INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0115 - accuracy: 0.9974 - val_loss: 0.0504 - val_accuracy: 0.9872
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0083 - accuracy: 0.9982INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


...

4000/4000 [==============================] - 161s 40ms/step - loss: 0.0040 - accuracy: 0.9992 - val_loss: 0.0665 - val_accuracy: 0.9875
Epoch 9/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0039 - accuracy: 0.9992INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.0737 - val_accuracy: 0.9868
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0038 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 0.0787 - val_accuracy: 0.9864


In [45]:
model_bin = tf.keras.models.load_model('best_model_acc_fase8_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (9,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns

    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 400s 13ms/step
Matriz de Confusão Teste:  9
[[485970  14030]
 [  7694 492306]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9844    0.9719    0.9781    500000
           1     0.9723    0.9846    0.9784    500000

    accuracy                         0.9783   1000000
   macro avg     0.9784    0.9783    0.9783   1000000
weighted avg     0.9784    0.9783    0.9783   1000000

____________________
31250/31250 [==============================] - 384s 12ms/step
Matriz de Confusão Teste:  10
[[485970  14030]
 [  8259 491741]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9833    0.9719    0.9776    500000
           1     0.9723    0.9835    0.9778    500000

    accuracy                         0.9777   1000000
   macro avg     0.9778    0.9777    0.9777   1000000
weighted avg     0.9778    0.9777    0.9777   1000000

__________________

## Décimo treino Incremental

In [46]:
model_bin = tf.keras.models.load_model('best_model_acc_fase8_balance')
del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

del(dga)
dga = pd.read_csv('./teste/df_teste9.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase9_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0202 - accuracy: 0.9948INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0202 - accuracy: 0.9948 - val_loss: 0.0419 - val_accuracy: 0.9872
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0115 - accuracy: 0.9974INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0115 - accuracy: 0.9974 - val_loss: 0.0471 - val_accuracy: 0.9877
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0083 - accuracy: 0.9982INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


...

4000/4000 [==============================] - 161s 40ms/step - loss: 0.0041 - accuracy: 0.9992 - val_loss: 0.0651 - val_accuracy: 0.9877
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0039 - accuracy: 0.9992INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.0724 - val_accuracy: 0.9866
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0039 - accuracy: 0.9992INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9_balance/assets


4000/4000 [==============================] - 162s 40ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.0933 - val_accuracy: 0.9828


In [47]:
model_bin = tf.keras.models.load_model('best_model_acc_fase9_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (10,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns

    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 391s 13ms/step
Matriz de Confusão Teste:  10
[[487646  12354]
 [  8942 491058]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9820    0.9753    0.9786    500000
           1     0.9755    0.9821    0.9788    500000

    accuracy                         0.9787   1000000
   macro avg     0.9787    0.9787    0.9787   1000000
weighted avg     0.9787    0.9787    0.9787   1000000

____________________
31250/31250 [==============================] - 376s 12ms/step
Matriz de Confusão Teste:  11
[[487646  12354]
 [  9827 490173]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9802    0.9753    0.9778    500000
           1     0.9754    0.9803    0.9779    500000

    accuracy                         0.9778   1000000
   macro avg     0.9778    0.9778    0.9778   1000000
weighted avg     0.9778    0.9778    0.9778   1000000

_________________

## Décimo primeiro treino Incremental

In [48]:
model_bin = tf.keras.models.load_model('best_model_acc_fase9_balance')
del(majestic)
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

del(dga)
dga = pd.read_csv('./teste/df_teste10.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

del(df_treino)
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase10_balance',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
4000/4000 [==============================] - ETA: 0s - loss: 0.0193 - accuracy: 0.9951INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


4000/4000 [==============================] - 164s 41ms/step - loss: 0.0193 - accuracy: 0.9951 - val_loss: 0.0319 - val_accuracy: 0.9902
Epoch 2/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0106 - accuracy: 0.9977INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0106 - accuracy: 0.9977 - val_loss: 0.0506 - val_accuracy: 0.9873
Epoch 3/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0075 - accuracy: 0.9984INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


...

4000/4000 [==============================] - 162s 40ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.0703 - val_accuracy: 0.9876
Epoch 8/10
4000/4000 [==============================] - 160s 40ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.0767 - val_accuracy: 0.9865
Epoch 9/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0037 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


4000/4000 [==============================] - 162s 41ms/step - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.0855 - val_accuracy: 0.9850
Epoch 10/10
3999/4000 [============================>.] - ETA: 0s - loss: 0.0035 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


INFO:tensorflow:Assets written to: best_model_acc_fase10_balance/assets


4000/4000 [==============================] - 161s 40ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.0847 - val_accuracy: 0.9846


In [49]:
model_bin = tf.keras.models.load_model('best_model_acc_fase10_balance')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (11,12):
    del(dga)
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns

    del(df_teste)
    df_teste = pd.concat([legit,dga.sample(500000, random_state=1)], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

31250/31250 [==============================] - 393s 13ms/step
Matriz de Confusão Teste:  11
[[487087  12913]
 [  8211 491789]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9834    0.9742    0.9788    500000
           1     0.9744    0.9836    0.9790    500000

    accuracy                         0.9789   1000000
   macro avg     0.9789    0.9789    0.9789   1000000
weighted avg     0.9789    0.9789    0.9789   1000000

____________________
